##雲端硬碟設定指向

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls 'drive'

MyDrive  Shareddrives


In [3]:
import os
path = "/content/drive/My Drive/Colab Notebooks/BERT_Spacy3/"
os.chdir(path)

In [4]:
!ls

 1.Ner_converter_tea.ipynb			     data_ner
 1.RelationBinary_converter_tea.ipynb		     data_relation
 2.Train_Spacy_Bert_FineTuneNER_tea.ipynb	     outputs
'2.Train_Spacy_Bert_Relation Extraction_tea.ipynb'   rel_component
 3.Predict_tea.ipynb				     原文參考
 configs_ner


In [5]:
# 安裝 spacy
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# UBIAI 註釋工具導出的 IOB 文件轉換為 spacy JSON
# https://spacy.io/api/cli#convert
!python -m spacy convert data_ner/train5.tsv ./data_ner -t json -n 1 -c iob
!python -m spacy convert data_ner/dev5.tsv ./data_ner -t json -n 1 -c iob

ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
⚠ No sentence boundaries found. Use `-s` to automatically segment
sentences.
✔ Generated output file (1 documents): data_ner/train5.json
ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
⚠ No sentence boundaries found. Use `-s` to automatically segment
sentences.
✔ Generated output file (1 documents): data_ner/dev5.json


In [7]:
# 將它們轉換為 spacy 二進製文件：
!python -m spacy convert data_ner/train5.json ./data_ner -t spacy
!python -m spacy convert data_ner/dev5.json ./data_ner -t spacy

✔ Generated output file (331 documents): data_ner/train5.spacy
✔ Generated output file (73 documents): data_ner/dev5.spacy


In [8]:
#!python -m spacy download en_core_web_trf
!python -m spacy download zh_core_web_trf # 中文版

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 417.4 MB 18 kB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
     |████████████████████████████████| 2.4 MB 29.4 MB/s 
     |████████████████████████████████| 6.6 MB 58.8 MB/s 
     |████████████████████████████████| 101 kB 13.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_trf')


In [9]:
# 安裝cuda：
# 為了加速訓練過程，需要在 GPU 上運行並行處理。為此安裝了 NVIDIA 9.2 cuda 庫：
# 要檢查是否安裝了正確的 cuda 編譯器，請運行：!nvcc --version
# 檢查檔案是否存在
import os
if os.path.isfile("cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb"):
  print("cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb檔案存在")
else:
  print("cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb檔案不存在")
  !wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb
!apt-key add /var/cuda-repo-9–2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

cuda-repo-ubuntu1604–9–2-local_9.2.88–1_amd64.deb檔案不存在
--2022-07-11 05:57:35--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.39.144
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.39.144|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2022-07-11 05:57:36--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?ZmzI4LWIJkP_3Q6K9k00IO8G0RZlST1MPRHV_IPZRuMSbtjmx_nACT

In [10]:
# 安裝為 cuda 9.2 配置的 pytorch
!pip install torch==1.7.1+cu92 torchvision==0.8.2+cu92 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 577.3 MB 3.3 kB/s 
     |████████████████████████████████| 12.5 MB 51.7 MB/s 
     |████████████████████████████████| 7.6 MB 32.9 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.11.0+cu113
    Uninstalling torchaudio-0.11.0+cu113:
      Successfully uninstalled torchaudio-0.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed

In [11]:
# Install spacy and transformer packages:
# pytorch 安裝後，需要安裝為 cuda 9.2 調整的 spacy 轉換器，並更改 CUDA_PATH 和 LD_LIBRARY_PATH 如下。最後，安裝相當於 numpy 庫但用於 GPU 的 cupy 庫：
!export CUDA_PATH="/usr/local/cuda-9.2"
!pip install -U spacy[cuda92,transformers]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55.0 MB 74 kB/s 


In [12]:
# Install cupy
!pip install cupy
# Set the cuda path
!export LD_LIBRARY_PATH=$CUDA_PATH/lib64:$LD_LIBRARY_PATH

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 35.5 MB/s 
  ERROR: Failed building wheel for cupy
  Running setup.py clean for cupy
  ERROR: Failed cleaning build dir for cupy
Failed to build cupy
    Running setup.py install for cupy ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-5p44xmzx/cupy_92bb501ff55e4f2a95d9ccfa779dd913/setup.py'"'"'; __file__='"'"'/tmp/pip-install-5p44xmzx/cupy_92bb501ff55e4f2a95d9ccfa779dd913/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-poaapy3f/install-record.txt --single-version-externally-manag

In [13]:
!python -m spacy debug data configs_ner/base_config.cfg

✘ Config validation error
disabled                  field required
tokenizer                 field required
before_creation           field required
after_creation            field required
after_pipeline_creation   field required

{'lang': 'zh', 'pipeline': ['transformer', 'ner'], 'batch_size': 128}

If your config contains missing values, you can run the 'init fill-config'
command to fill in all the defaults, if possible:

python -m spacy init fill-config configs_ner/base_config.cfg configs_ner/base_config.cfg 



In [14]:
# https://spacy.io/usage/training 選擇設定值下載設定檔文件
# 在使用訓練和測試路徑更新 spacy config.cfg 文件後，我們使用 BERT 模型所需的其餘參數自動填充配置文件
!python -m spacy init fill-config configs_ner/base_config.cfg configs_ner/config_spacy.cfg

✔ Auto-filled config with all values
✔ Saved config
configs_ner/config_spacy.cfg
You can now add your data and train your pipeline:
python -m spacy train config_spacy.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [15]:
# Before launching the training, lets debug the config file to make sure everything is correct:
!python -m spacy debug data configs_ner/config_spacy.cfg


============================ Data file validation ============================
Downloading: 100% 29.0/29.0 [00:00<00:00, 26.2kB/s]
Downloading: 100% 624/624 [00:00<00:00, 355kB/s]
Downloading: 100% 107k/107k [00:00<00:00, 160kB/s] 
Downloading: 100% 263k/263k [00:00<00:00, 308kB/s]
Downloading: 100% 393M/393M [00:05<00:00, 69.4MB/s]
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializi

In [16]:
!pip uninstall cupy

In [17]:
!pip install cupy-cuda100

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58.9 MB 1.2 MB/s 


In [18]:
from datetime import datetime
result = datetime.now().strftime("%Y-%m-%d %H:%M:%S %p")
print(result)

2022-07-11 06:03:38 AM


In [19]:
#Finally, we are ready to start the training:
!python -m spacy train -g 0 configs_ner/config_spacy.cfg --output ./

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-07-11 06:03:46,392] [INFO] Set up nlp object from config
[2022-07-11 06:03:46,404] [INFO] Pipeline: ['transformer', 'ner']
[2022-07-11 06:03:46,408] [INFO] Created vocabulary
[2022-07-11 06:03:46,409] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This 

In [20]:
import spacy
nlp = spacy.load("model-best")
text = [
'''
至於苗栗縣茶區的部分，早期種植紅茶、綠茶、烏龍茶、包種茶……等等，近幾年來，因外銷的不景氣，而改製內銷的茶業，包括頭屋、頭份一帶的明德茶(舊名老田寮茶)、 獅潭鄉的仙山茶、造橋鄉的龍鳳茶、大湖鄉的巖茶等，老田寮茶的改名是因為茶區附近的明德水庫興建。而為了利於推廣，把明德茶、仙山茶、龍鳳茶、巖茶，同類型的統稱為苗栗烏龍茶，福壽茶、白毛烏龍茶、東方美人茶，統稱苗栗椪風茶。

南投則是全台唯一全縣13個鄉鎮幾乎都有生產茶，各鄉鎮因氣候上的不同，及做工上的差異，所生產的茶葉各具特色，大致可分為手採茶區及機械採茶區兩大類。

以人工手採為主的有玉山烏龍茶、凍頂烏龍茶及霧社廬山烏龍茶。玉山烏龍茶茶業外型緊結身骨重，沖泡後茶湯城清澈蜜綠色，香氣幽雅，滋味濃厚甘醇，即使是夏茶亦不帶苦澀味，為高海拔玉山烏龍茶的特色。

鹿谷凍頂烏龍茶，也稱為凍頂茶，生產自南投縣鹿谷鄉的凍頂村，主要是以青心烏龍為原料製成的半球形包種茶。其中在製茶過程中，「團揉」為製造凍頂茶獨特的「中國功夫」技法，茶菁烘乾後，需再重複以布包成球狀揉捻茶葉，使茶成半發酵半球狀，稱為「布揉製茶」或「熱團揉」，茶葉形狀條鎖緊節整齊，葉片捲曲成蝦球狀，茶湯水色呈金黃且澄清明亮，落喉韻味強且經久耐泡為凍頂茶之特色。

霧社廬山烏龍茶，生產於仁愛鄉 ，由於天然條件優厚,所產製的茶葉清香甘醇。又鄉內有許多名勝風景區山明水秀、風光明媚，為南投縣內最具發展潛力的高山茶區。

清境宿霧茶為南投縣仁愛鄉清境著名的高山茶，因地勢較高，且長年籠罩在雲霧中就此得名。素有「霧上桃源」之稱的清境，海拔超過1,600公尺氣候宜人，所以生產出來的茶葉彷彿是沉睡在嵐霧中，接受雲霧的滋潤，風味口感絕佳，頗受各界歡迎。

以機械採茶的松柏長青茶，原名為「埔中茶」或稱「松柏坑茶」，松柏坑茶在台灣的茶業發展史上開發極早，在西元1975年時任行政院長蔣經國先生(蒞臨巡視，對此地茶葉香郁芬芳稱讚不已，特命名為「松柏長青茶」。

生產於南投縣魚池鄉的魚池鄉日月潭紅茶，在日治時代由印度阿薩姆省引進的大葉種茶種，曾經以「Formosa Black Tea」為名，在英國倫敦得到很高的評價，曾為日本天皇御用貢品。民國67年「魚池鄉紅茶產業文化促進協會」並自創「阿薩姆紅茶」品牌，堅持手工摘製風味。
'''
]
for doc in nlp.pipe(text, disable=["tagger", "parser"]):
    print([(ent.text, ent.label_) for ent in doc.ents])

[('苗栗縣', '地點'), ('綠茶', '茶名'), ('烏龍茶', '茶名'), ('包種茶', '茶名'), ('…', '茶樹名'), ('…', '茶樹名'), ('獅潭鄉', '地點'), ('仙山茶', '茶名'), ('造橋鄉', '茶名'), ('龍鳳茶', '茶名'), ('大湖鄉', '地點'), ('明德茶', '茶名'), ('龍鳳茶', '茶名'), ('苗栗', '地點'), ('福壽茶', '茶名'), ('白毛烏龍茶', '茶名'), ('東方美人茶', '茶名'), ('苗栗', '地點'), ('椪風茶', '茶名'), ('玉山烏龍茶', '茶名'), ('凍頂烏龍茶', '茶名'), ('霧社廬山烏龍茶', '茶名'), ('玉山烏龍茶', '茶名'), ('鹿谷', '地點'), ('凍頂烏龍茶', '茶名'), ('凍頂茶', '茶名'), ('南投縣鹿谷', '地點'), ('凍頂村', '茶名'), ('霧社', '茶名'), ('仁愛鄉', '地點'), ('南投縣魚池鄉', '地點'), ('阿薩姆省', '地點'), ('F', '茶樹名'), ('a', '茶樹名'), ('B', '茶樹名'), ('l', '茶樹名'), ('c', '茶樹名'), ('k', '茶樹名'), ('T', '茶樹名'), ('e', '茶樹名'), ('a', '茶樹名'), ('民國67年「魚池鄉紅茶產業文化促進協會」並自創「阿薩姆紅茶」品牌，堅持手工摘製風味。\n', '時間')]


In [21]:
from datetime import datetime
result = datetime.now().strftime("%Y-%m-%d %H:%M:%S %p")
print(result)

2022-07-11 06:35:57 AM
